# Olist Brazilian E-Commerce - Merging Datasets and Consistency Checks

### This script contains the following:

#### 1.   Import Libraries
#### 2.   Import Order Reviews Data Set
#### 3.   Import Orders-Items-Products Data Set
#### 4.   Merge: Orders_Items_Products with Order_reviews Data Set
#### 5.   Import Customers Data Set 
#### 6.   Merge: Orders_Items_Products_Reviews with Customers Data Set
#### 7.   Import Sellers Data Set
#### 8.   Merge: Orders_Items_Products_Reviews_Customers with Sellers Data Set


### 1. Import Libraries

In [3]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
import folium
import json

In [6]:
# the charts created are displayed in the notebook without the need to "call" them specifically.

%matplotlib inline

In [7]:
# define a path
path = r'C:\Users\junio\OneDrive\Career Foundry Project\Achievement 6\02-2024 Brazilian E-Commerce'

### 2. Import Data sets 

### 2.1. Order Reviews Data Set

In [13]:
reviews = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'olist_order_reviews_wrangled.csv'), index_col = False)

In [ ]:
reviews.head()

In [ ]:
reviews.describe()

### Remaining cleaning procedures before merging with other datasets

#### Dropping Columns:

In [15]:
# Drop unnecessary columns  from reviews dataframe

reviews.drop(columns="Unnamed: 0", inplace=True)

In [16]:
reviews.shape

(98997, 4)

In [ ]:
reviews.columns

#### Renaming Columns:

In [21]:
# Rename columns 

reviews.rename(columns = {'review_comment_title' : 'review_title', 'review_comment_message': 'review_comment' }, inplace = True)

#### Missing Values Check:

In [ ]:
# Check for missing values

reviews.isnull().sum() # we cannot assure wheter these missing values are missing or are just zeros

#### Duplicates Check:

In [24]:
# Find duplicates

reviews_dups = reviews[reviews.duplicated()]

In [26]:
reviews_dups.shape # No Duplicates!

(0, 4)

#### Mixed_type Data:

In [27]:
for col in reviews.columns.tolist():
      weird = (reviews[[col]].applymap(type) != reviews[[col]].iloc[0].apply(type)).any(axis = 1)
      if len(reviews[weird]) > 0:
        print(col) 
      else:
         print("No mixed-type columns found!")   

No mixed-type columns found!
No mixed-type columns found!
review_title
review_comment


C:\Users\junio\AppData\Local\Temp\ipykernel_4564\1666836673.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (reviews[[col]].applymap(type) != reviews[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\junio\AppData\Local\Temp\ipykernel_4564\1666836673.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (reviews[[col]].applymap(type) != reviews[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\junio\AppData\Local\Temp\ipykernel_4564\1666836673.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (reviews[[col]].applymap(type) != reviews[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\junio\AppData\Local\Temp\ipykernel_4564\1666836673.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (reviews[[col]].applymap(type) != reviews[[col]].iloc[0].apply(type)).any(axis = 1)


In [29]:
# Convert values in 'review_title' column to strings
reviews['review_title'] = reviews['review_title'].astype(str)

# Convert values in 'review_comment' column to strings
reviews['review_comment'] = reviews['review_comment'].astype(str)

In [30]:
# check if all values are indeed strings

print(reviews['review_title'].dtype)
print(reviews['review_comment'].dtype)

object
object


### 3. Import Orders-Items-Products Data Set

In [34]:
# Import data set that was merged in the previous task, it is already cleaned!

ords_items_prods = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'ords_items_prod_merged_price_flag.csv'), index_col = False)

In [ ]:
ords_items_prods.head()

In [37]:
# Drop unnecessary columns  from reviews dataframe

ords_items_prods.drop(columns="Unnamed: 0", inplace=True)

### 4. Merge: Orders_Items_Products with Order_reviews Data Set
       unique identifier : order_id

In [43]:
ords_items_prods_reviews = ords_items_prods.merge(reviews, on = 'order_id', indicator = True)

In [ ]:
ords_items_prods_reviews.head()

In [45]:
# check whether there is a full match or not. 
### They do have a full match  the default Join was applied (using how = 'inner')., tha is, Inner Join)

ords_items_prods_reviews['_merge'].value_counts(dropna = False)   # The merge worked!

_merge
both          108198
left_only          0
right_only         0
Name: count, dtype: int64

In [47]:
# Drop the _merge flag

ords_items_prods_reviews.drop(columns='_merge', inplace=True)

In [49]:
# Export this merged version of the data

ords_items_prods_reviews.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'ords_items_prods_reviews_merged.csv'), index = False)

### 5. Import Customers Data Set 

In [63]:
customers = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'olist_customers_wrangled.csv'), index_col = False)

In [ ]:
customers.head()

In [69]:
customers.shape

(99441, 5)

### Remaining cleaning procedures before merging with other datasets

#### Dropping Columns

In [68]:
# Drop unnecessary columns  from reviews dataframe

customers.drop(columns="Unnamed: 0", inplace=True)

#### Missing Values Checks:

In [ ]:
# Check for missing values

customers.isnull().sum()  # No Missing Values!

#### Duplicates Checks:

In [71]:
# Find duplicates

customers_dups = customers[customers.duplicated()]

In [72]:
customers_dups.shape # No Diplicates either!

(0, 5)

#### Mixed_type Data:

In [ ]:
for col in customers.columns.tolist():
      weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
      if len(customers[weird]) > 0:
        print(col) 
      else:
         print("No mixed-type columns found!")   ### No mixed-type columns!

### 6. Merge: Orders_Items_Products_Reviews with Customers Data Set
       unique identifier: customer_id

In [77]:
# merge
ords_items_prods_reviews_customers = ords_items_prods_reviews.merge(customers, on = 'customer_id', indicator = True)

In [85]:
# check head

ords_items_prods_reviews_customers.head()

,order_id,customer_id,purchase_date,delivered_customer_date,estimated_delivery_date,item_count,product_id,seller_id,price,freight_value,prod_category,Price category,review_score,review_title,review_comment,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,housewares,Low price,4,nan,"Não testei o produto ainda, mas ele veio corre...",7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,2017-08-18 14:44:43,2017-08-28 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,housewares,Low price,4,nan,Deveriam embalar melhor o produto. A caixa vei...,3a51803cc0d012c3b5dc8b7528cb05f7,3366,sao paulo,SP
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,2017-08-07 18:30:01,2017-08-15 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,housewares,Low price,5,nan,"Só achei ela pequena pra seis xícaras ,mais é ...",ef0996a1a279c26e7ecbd737be23d235,2290,sao paulo,SP
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,2017-11-07 18:04:59,2017-11-13 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,housewares,Low price,3,nan,nan,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC
4,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,22.76,perfumery,Middle price,4,Muito boa a loja,Muito bom o produto.,af07308b275d755c9edb36a90c618231,47813,barreiras,BA


In [ ]:
# check whether there is a full match or not. 

ords_items_prods_reviews_customers['_merge'].value_counts(dropna = False)   # The merge worked!

In [80]:
# Drop the _merge flag

ords_items_prods_reviews_customers.drop(columns='_merge', inplace=True)

In [81]:
# Export this merged version of the data

ords_items_prods_reviews_customers.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'ords_items_prods_reviews_customers_merged.csv'), index = False)

In [ ]:
ords_items_prods_reviews_customers.describe()

In [ ]:
ords_items_prods_reviews_customers.info()

### 7. Import Sellers Data Set 

In [57]:
sellers = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'olist_sellers_wrangled.csv'), index_col = False)

In [61]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


### Remaining cleaning procedures before merging with other datasets

#### Dropping Columns:

In [59]:
# Drop unnecessary columns  from reviews dataframe

sellers.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
sellers.info()

### 8.   Merge: Orders_Items_Products_Reviews_Customers with Sellers Data Set
         unique identifier: seller_id

In [87]:
# merge 
ords_items_prods_reviews_customers_sellers = ords_items_prods_reviews_customers.merge(sellers, on = 'seller_id', indicator = True)

In [96]:
ords_items_prods_reviews_customers_sellers.shape

(108198, 22)

In [ ]:
# check whether there is a full match or not. 

ords_items_prods_reviews_customers_sellers['_merge'].value_counts(dropna = False)   # The merge worked!

In [97]:
# check head

ords_items_prods_reviews_customers_sellers.head()

,order_id,customer_id,purchase_date,delivered_customer_date,estimated_delivery_date,item_count,product_id,seller_id,price,freight_value,prod_category,Price category,review_score,review_title,review_comment,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,housewares,Low price,4,nan,"Não testei o produto ainda, mas ele veio corre...",7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,9350,maua,SP
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,2017-08-18 14:44:43,2017-08-28 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,housewares,Low price,4,nan,Deveriam embalar melhor o produto. A caixa vei...,3a51803cc0d012c3b5dc8b7528cb05f7,3366,sao paulo,SP,9350,maua,SP
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,2017-08-07 18:30:01,2017-08-15 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,housewares,Low price,5,nan,"Só achei ela pequena pra seis xícaras ,mais é ...",ef0996a1a279c26e7ecbd737be23d235,2290,sao paulo,SP,9350,maua,SP
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,2017-11-07 18:04:59,2017-11-13 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,housewares,Low price,3,nan,nan,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,9350,maua,SP
4,8736140c61ea584cb4250074756d8f3b,ab8844663ae049fda8baf15fc928f47f,2017-08-10 13:35:55,2017-08-16 19:03:36,2017-08-23 00:00:00,1,b00a32a0b42fd65efb58a5822009f629,3504c0cb71d7fa48d967e0e4c94d59d9,75.90,7.79,baby,Middle price,5,nan,"custo beneficio, simples de usar e rápido",02c9e0c05a817d4562ec0e8c90f29dba,8577,itaquaquecetuba,SP,9350,maua,SP


In [92]:
# Set display options to show all rows and columns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [95]:
# Drop merge flag

ords_items_prods_reviews_customers_sellers.drop(columns = '_merge', inplace = True)

In [ ]:
ords_items_prods_reviews_customers_sellers.info()

In [102]:
# Export this merged version of the data set

ords_items_prods_reviews_customers_sellers.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'ords_items_prods_reviews_cust_sellers_merged.csv'), index = False)